In [1]:
import httplib2
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import webbrowser
import re

In [2]:
url = "https://www.quebec.ca/sante/problemes-de-sante/a-z/coronavirus-2019/situation-coronavirus-quebec/"

In [3]:
resp = requests.get(url)

In [4]:
# resp

In [5]:
soup = BeautifulSoup(resp.text, "lxml")

In [6]:
# type(soup)

In [7]:
# pprint(soup)

In [8]:
table = soup.find("table")

In [9]:
# print(table.prettify())

In [10]:
# p_tag_in_table = table.find_all("p")

In [11]:
def get_p_tags(data):
    return [str(i) for i in data.find_all("p")]

In [12]:
confirmes_p_tags = get_p_tags(table)

In [13]:
confirmes_p_tags

['<p>Régions</p>',
 '<p class="texteCentre">Nombre de cas confirmés, en date du 3\xa0juin, 11\xa0h</p>',
 '<p>Données cumulatives. Source\xa0: V10, MSSS, 2\xa0juin\xa02020, 18\xa0h</p>',
 '<p>01 - Bas-Saint-Laurent</p>',
 '<p class="texteCentre">48</p>',
 '<p>02 - Saguenay – Lac-Saint-Jean</p>',
 '<p class="texteCentre">321</p>',
 '<p>03 - Capitale-Nationale</p>',
 '<p class="texteCentre">1\xa0630</p>',
 '<p>04 - Mauricie-et-Centre-du-Québec</p>',
 '<p class="texteCentre">1\xa0990</p>',
 '<p>05 - Estrie</p>',
 '<p class="texteCentre">943</p>',
 '<p>06 - Montréal</p>',
 '<p class="texteCentre">25\xa0788</p>',
 '<p>07 - Outaouais</p>',
 '<p class="texteCentre">547</p>',
 '<p>08 - Abitibi-Témiscamingue</p>',
 '<p class="texteCentre">168</p>',
 '<p>09 - Côte-Nord</p>',
 '<p class="texteCentre">115</p>',
 '<p>10 - Nord-du-Québec</p>',
 '<p class="texteCentre">8</p>',
 '<p>11 - Gaspésie-Îles-de-la-Madeleine</p>',
 '<p class="texteCentre">184</p>',
 '<p>12 - Chaudière-Appalaches</p>',
 '<p cl

In [14]:
def find_line(d):
    l = []
    for i in range(len(d)):
        if d[i].startswith("<p>Données cumulatives"):
            l.append(d[i])
    return l[0]

In [15]:
date_str = find_line(confirmes_p_tags)

In [16]:
date_str

'<p>Données cumulatives. Source\xa0: V10, MSSS, 2\xa0juin\xa02020, 18\xa0h</p>'

In [17]:
date_str_ignore_ascii = date_str.encode("ascii", "replace")

In [18]:
date_str_ignore_ascii

b'<p>Donn?es cumulatives. Source?: V10, MSSS, 2?juin?2020, 18?h</p>'

In [19]:
date_str_decode_utf8 = date_str_ignore_ascii.decode("UTF8")

In [20]:
date_str_decode_utf8

'<p>Donn?es cumulatives. Source?: V10, MSSS, 2?juin?2020, 18?h</p>'

In [21]:
date_str_regex = re.compile("\d{1,2}\W\w+\W\d{4}")

In [22]:
date_str_regex

re.compile(r'\d{1,2}\W\w+\W\d{4}', re.UNICODE)

In [23]:
date_found = date_str_regex.findall(date_str_decode_utf8)

In [24]:
date_found

['2?juin?2020']

In [25]:
date = " ".join(date_found[0].split("?"))

In [26]:
date

'2 juin 2020'

In [27]:
 donnees_confirmes = confirmes_p_tags[3:]

In [28]:
donnees_confirmes

['<p>01 - Bas-Saint-Laurent</p>',
 '<p class="texteCentre">48</p>',
 '<p>02 - Saguenay – Lac-Saint-Jean</p>',
 '<p class="texteCentre">321</p>',
 '<p>03 - Capitale-Nationale</p>',
 '<p class="texteCentre">1\xa0630</p>',
 '<p>04 - Mauricie-et-Centre-du-Québec</p>',
 '<p class="texteCentre">1\xa0990</p>',
 '<p>05 - Estrie</p>',
 '<p class="texteCentre">943</p>',
 '<p>06 - Montréal</p>',
 '<p class="texteCentre">25\xa0788</p>',
 '<p>07 - Outaouais</p>',
 '<p class="texteCentre">547</p>',
 '<p>08 - Abitibi-Témiscamingue</p>',
 '<p class="texteCentre">168</p>',
 '<p>09 - Côte-Nord</p>',
 '<p class="texteCentre">115</p>',
 '<p>10 - Nord-du-Québec</p>',
 '<p class="texteCentre">8</p>',
 '<p>11 - Gaspésie-Îles-de-la-Madeleine</p>',
 '<p class="texteCentre">184</p>',
 '<p>12 - Chaudière-Appalaches</p>',
 '<p class="texteCentre">500</p>',
 '<p>13 - Laval</p>',
 '<p class="texteCentre">5\xa0488</p>',
 '<p>14 - Lanaudière</p>',
 '<p class="texteCentre">4\xa0037</p>',
 '<p>15 - Laurentides</p>',
 '

#### Slice Regions

In [29]:
slice_regions = donnees_confirmes[0::2]

In [30]:
# slice_regions

#### Regex Régions

In [31]:
def regex_regions(r):
    regex = [re.findall(r"\w+", noms) for noms in r]
    les_noms = []
    regex_tranches = [i for i in regex]
    for m in regex_tranches:
        if len(m) > 1:
            nom = "-".join(m)
            les_noms.append(nom)
        else:
            les_noms.append(m[0])
    les_noms_2 = []
    for i in les_noms:
        les_noms_2.append(re.sub("p-\d.-|p-|-p", "", i))
    return les_noms_2        

In [32]:
regions = regex_regions(slice_regions)

In [33]:
# regions

#### Slice Confirmés

In [34]:
# donnees_confirmes[1::2]

In [35]:
slice_confirmes = donnees_confirmes[1::2]

In [36]:
# slice_confirmes

#### Regex Nombres

In [37]:
def nombre_de_cas(c):
    if len(c) == 1:
        regions_a_determine = []
        regex = re.findall(r"\d+", c[0])
        regions_a_determine = int(regex[0])
        return regions_a_determine
    else:
        les_cas = []
        for i in c:
            chiffre_regex = re.findall(r"\d+", i)
            les_cas.append(int("".join(chiffre_regex)))
        return les_cas

In [38]:
confirmes = nombre_de_cas(donnees_confirmes[1::2]) # error

In [39]:
confirmes

[48,
 321,
 1630,
 1990,
 943,
 25788,
 547,
 168,
 115,
 8,
 184,
 500,
 5488,
 4037,
 2984,
 7091,
 16,
 10,
 13,
 3,
 51884]

#### Régions Aleat

In [40]:
# # total 21
# len(confirmes)

### Cas de Décès

In [41]:
soup_deces = soup.find_all("table")

In [42]:
# len(soup_deces)

In [43]:
# soup_deces

In [44]:
donnees_deces = soup_deces[2]

In [45]:
donnees_deces

<table class="contenttable"><thead><tr><th class="align-top"><p>Régions</p></th><th><p class="texteCentre">Nombre de décès, en date du 3 juin, 11 h</p></th></tr></thead><tfoot><tr><td colspan="2"><p>Données cumulatives. Source : V10, MSSS, 2 juin 2020, 18 h</p></td></tr></tfoot><tbody><tr><td><p>01 - Bas-Saint-Laurent</p></td><td><p class="texteCentre">2</p></td></tr><tr><td><p>02 - Saguenay – Lac-Saint-Jean</p></td><td><p class="texteCentre">26</p></td></tr><tr><td><p>03 - Capitale-Nationale</p></td><td><p class="texteCentre">128</p></td></tr><tr><td><p>04 - Mauricie-et-Centre-du-Québec</p></td><td><p class="texteCentre">203</p></td></tr><tr><td><p>05 - Estrie</p></td><td><p class="texteCentre">25</p></td></tr><tr><td><p>06 - Montréal</p></td><td><p class="texteCentre">2 982</p></td></tr><tr><td><p>07 - Outaouais</p></td><td><p class="texteCentre">22</p></td></tr><tr><td><p>08 - Abitibi-Témiscamingue</p></td><td><p class="texteCentre">3</p></td></tr><tr><td><p>09 - Côte-Nord</p></td><

In [46]:
p_tag_deces = get_p_tags(donnees_deces)
# p_tag_deces = [str(i) for i in donnees_deces.find_all("p")]

In [47]:
# p_tag_deces[4::2]

In [48]:
# len(p_tag_deces[1::2])

In [49]:
deces = nombre_de_cas(p_tag_deces[4::2])

In [50]:
# deces

In [51]:
# len(deces)

In [52]:
donnees_covid = []
for i in range(len(confirmes)):
    donnees_covid.append(dict(confirmes=confirmes[i], date=date,
                  deces=deces[i],  region=regions[i]))

In [53]:
 data = donnees_covid 

In [54]:
data

[{'confirmes': 48,
  'date': '2 juin 2020',
  'deces': 2,
  'region': 'Bas-Saint-Laurent'},
 {'confirmes': 321,
  'date': '2 juin 2020',
  'deces': 26,
  'region': 'Saguenay-Lac-Saint-Jean'},
 {'confirmes': 1630,
  'date': '2 juin 2020',
  'deces': 128,
  'region': 'Capitale-Nationale'},
 {'confirmes': 1990,
  'date': '2 juin 2020',
  'deces': 203,
  'region': 'Mauricie-et-Centre-du-Québec'},
 {'confirmes': 943, 'date': '2 juin 2020', 'deces': 25, 'region': 'Estrie'},
 {'confirmes': 25788,
  'date': '2 juin 2020',
  'deces': 2982,
  'region': 'Montréal'},
 {'confirmes': 547, 'date': '2 juin 2020', 'deces': 22, 'region': 'Outaouais'},
 {'confirmes': 168,
  'date': '2 juin 2020',
  'deces': 3,
  'region': 'Abitibi-Témiscamingue'},
 {'confirmes': 115, 'date': '2 juin 2020', 'deces': 0, 'region': 'Côte-Nord'},
 {'confirmes': 8,
  'date': '2 juin 2020',
  'deces': 0,
  'region': 'Nord-du-Québec'},
 {'confirmes': 184,
  'date': '2 juin 2020',
  'deces': 8,
  'region': 'Gaspésie-Îles-de-la-Ma